In [1]:
pip install langchain openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain langchain-community

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool

llm = ChatOpenAI(temperature=0.7)

/var/folders/8p/lhfbj_0n501d3lhdqz4xc0dh0000gn/T/ipykernel_10247/4091910914.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7)


In [8]:

@tool
def suggest_travel_destinations(interests: str) -> str:
    """
    Suggest travel destinations based on user's interests such as beach, culture, adventure, or nature.
    """
    suggestions = {
        "beach": "Maldives, Bali, Hawaii",
        "culture": "Rome, Kyoto, Istanbul",
        "adventure": "New Zealand, Patagonia",
        "nature": "Iceland, Banff, Norway"
    }
    results = []
    for interest in interests.split(","):
        interest = interest.strip().lower()
        match = suggestions.get(interest, "No destinations found for this interest.")
        results.append(f"{interest.capitalize()}: {match}")
    return "\n".join(results)

In [10]:
@tool
def get_weather(destination: str) -> str:
    """
    Returns a simulated weather report for a given destination.
    """
    weather_data = {
        "Bali": "Sunny, 30°C",
        "Maldives": "Clear skies, 31°C",
        "Hawaii": "Partly cloudy, 29°C",

        "Rome": "Cloudy, 22°C",
        "Kyoto": "Rainy, 18°C",
        "Istanbul": "Sunny, 24°C",

        "New Zealand": "Windy, 15°C",
        "Patagonia": "Cold, 10°C",
        "Tokyo": "Humid, 27°C",

        "Iceland": "Snowy, -2°C",
        "Banff": "Chilly, 5°C",
        "Norway": "Cool and foggy, 12°C"
    }

    return weather_data.get(destination, "Weather data not available.")

In [11]:
@tool
def suggest_activities(destination: str) -> str:
    """
    Suggest popular activities to do in a specific destination.
    """
    activity_data = {
        "Bali": "Surfing, Yoga retreats, Beach walks",
        "Maldives": "Snorkeling, Spa days, Luxury resorts",
        "Hawaii": "Volcano hikes, Luaus, Surf lessons",

        "Rome": "Visit the Colosseum, Eat gelato, Walk in Trastevere",
        "Kyoto": "Visit temples, Tea ceremonies, Cherry blossom viewing",
        "Istanbul": "Explore Hagia Sophia, Cruise the Bosphorus, Try Turkish delights",

        "New Zealand": "Skydiving, Hiking, Bungee jumping",
        "Patagonia": "Glacier trekking, Mountain climbing, Nature photography",
        "Tokyo": "Robot cafes, Shibuya crossing, Sushi classes",

        "Iceland": "See the Northern Lights, Hot springs, Waterfall tours",
        "Banff": "Hiking, Canoeing, Wildlife watching",
        "Norway": "Fjord cruises, Mountain hiking, Northern Lights safaris"
    }

    return activity_data.get(destination, "No activity data available.")

In [12]:
# 6. Combine all tools into a list
tools = [suggest_travel_destinations, get_weather, suggest_activities]

In [13]:
# 7. Initialize the agent with the tools and model
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [14]:
response = agent.run("Suggest a destination for beach lovers.")
print(response)



> Entering new AgentExecutor chain...
I should use the suggest_travel_destinations tool to find a destination for beach lovers.
Action: suggest_travel_destinations
Action Input: "beach"
Observation: Beach: Maldives, Bali, Hawaii
Thought:I should check the weather for these beach destinations to make a decision.
Action: get_weather
Action Input: "Maldives"
Observation: Clear skies, 31°C
Thought:Maldives seems like a great beach destination with clear skies and warm weather.
Final Answer: Maldives

> Finished chain.
Maldives


In [16]:
response = agent.run("What's the weather in Istanbul?")
print(response)



> Entering new AgentExecutor chain...
I should use the get_weather tool to find out the weather in Istanbul.
Action: get_weather
Action Input: Istanbul
Observation: Sunny, 24°C
Thought:I now know the weather in Istanbul.
Final Answer: The weather in Istanbul is sunny with a temperature of 24°C.

> Finished chain.
The weather in Istanbul is sunny with a temperature of 24°C.


In [17]:
response = agent.run("What can I do in Bali?")
print(response)



> Entering new AgentExecutor chain...
I should find out popular activities to do in Bali.
Action: suggest_activities
Action Input: "Bali"
Observation: Surfing, Yoga retreats, Beach walks
Thought:I should also check the weather in Bali to see if it's a good time to visit.
Action: get_weather
Action Input: "Bali"
Observation: Sunny, 30°C
Thought:Based on the activities and weather in Bali, I can suggest travel destinations for someone interested in beaches and warm weather.
Action: suggest_travel_destinations
Action Input: "beach, warm weather"
Observation: Beach: Maldives, Bali, Hawaii
Warm weather: No destinations found for this interest.
Thought:I now know the final answer
Final Answer: Popular activities to do in Bali include surfing, yoga retreats, and beach walks. The weather is currently sunny at 30°C. For someone interested in beaches and warm weather, suggested travel destinations are Maldives, Bali, and Hawaii.

> Finished chain.
Popular activities to do in Bali include surfi